# Loop 61 Analysis: CQR Failed, What's Next?

**Latest Experiment**: CQR (Conformalized Quantile Regression) CV 0.009899 - 20.8% WORSE than best

**Key Questions**:
1. Why did CQR fail?
2. What approaches haven't been tried?
3. How can we change the CV-LB relationship?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Submission history
submissions = [
    ('exp_000', 0.011081, 0.0982),
    ('exp_001', 0.012297, 0.1065),
    ('exp_003', 0.010501, 0.0972),
    ('exp_005', 0.010430, 0.0969),
    ('exp_006', 0.009749, 0.0946),
    ('exp_007', 0.009262, 0.0932),
    ('exp_009', 0.009192, 0.0936),
    ('exp_012', 0.009004, 0.0913),
    ('exp_024', 0.008689, 0.0893),
    ('exp_026', 0.008465, 0.0887),
    ('exp_030', 0.008298, 0.0877),
    ('exp_041', 0.009002, 0.0932),
    ('exp_042', 0.014503, 0.1147),
]

df = pd.DataFrame(submissions, columns=['exp_id', 'cv', 'lb'])
print('Submission History:')
print(df.to_string(index=False))
print(f'\nBest CV: {df["cv"].min():.6f} ({df.loc[df["cv"].idxmin(), "exp_id"]})')
print(f'Best LB: {df["lb"].min():.4f} ({df.loc[df["lb"].idxmin(), "exp_id"]})')
print(f'Target: 0.0347')

In [ ]:
# Fit CV-LB relationship
slope, intercept, r_value, p_value, std_err = stats.linregress(df['cv'], df['lb'])
print(f'\nCV-LB Relationship:')
print(f'LB = {slope:.4f} × CV + {intercept:.4f}')
print(f'R² = {r_value**2:.4f}')
print(f'\nIntercept ({intercept:.4f}) vs Target (0.0347)')
if intercept > 0.0347:
    print(f'CRITICAL: Intercept > Target by {(intercept - 0.0347)/0.0347*100:.1f}%')
    print('CV minimization alone CANNOT reach target!')
else:
    print('Intercept < Target - CV minimization could work')

In [ ]:
# What CV would be needed to hit target?
required_cv = (0.0347 - intercept) / slope
print(f'\nRequired CV to hit target: {required_cv:.6f}')
if required_cv < 0:
    print('IMPOSSIBLE: Required CV is negative!')
    print('We need to CHANGE the CV-LB relationship, not just minimize CV')
else:
    print(f'Current best CV: {df["cv"].min():.6f}')
    print(f'Gap: {(df["cv"].min() - required_cv)/required_cv*100:.1f}%')

In [ ]:
# Analyze residuals - which experiments generalize better?
df['predicted_lb'] = slope * df['cv'] + intercept
df['residual'] = df['lb'] - df['predicted_lb']
df['residual_pct'] = df['residual'] / df['predicted_lb'] * 100

print('\nGeneralization Analysis (negative residual = better than expected):')
print(df[['exp_id', 'cv', 'lb', 'predicted_lb', 'residual', 'residual_pct']].sort_values('residual_pct').to_string(index=False))

print(f'\nBest generalizer: {df.loc[df["residual_pct"].idxmin(), "exp_id"]} (residual {df["residual_pct"].min():.1f}%)')
print(f'Worst generalizer: {df.loc[df["residual_pct"].idxmax(), "exp_id"]} (residual {df["residual_pct"].max():.1f}%)')

In [ ]:
# Plot CV-LB relationship
plt.figure(figsize=(10, 6))
plt.scatter(df['cv'], df['lb'], c='blue', s=100, alpha=0.7, label='Submissions')

# Fit line
cv_range = np.linspace(0, df['cv'].max() * 1.1, 100)
lb_pred = slope * cv_range + intercept
plt.plot(cv_range, lb_pred, 'r--', label=f'LB = {slope:.2f}×CV + {intercept:.4f}')

# Target line
plt.axhline(y=0.0347, color='green', linestyle=':', linewidth=2, label='Target (0.0347)')

# Intercept
plt.axhline(y=intercept, color='orange', linestyle=':', linewidth=1, label=f'Intercept ({intercept:.4f})')

plt.xlabel('CV Score')
plt.ylabel('LB Score')
plt.title('CV-LB Relationship Analysis')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/home/code/exploration/cv_lb_relationship.png', dpi=150)
plt.show()
print('\nSaved: /home/code/exploration/cv_lb_relationship.png')

In [ ]:
# What experiments have been tried?
experiments_tried = [
    ('MLP baseline', 'exp_000', 0.011081, 'Baseline'),
    ('LightGBM', 'exp_001', 0.012297, 'Worse'),
    ('DRFP + PCA', 'exp_002', 0.016948, 'Much worse'),
    ('Spange + DRFP', 'exp_003', 0.010501, 'Better'),
    ('Deep Residual', 'exp_004', 0.051912, 'Failed'),
    ('Large Ensemble (15)', 'exp_005', 0.010430, 'Marginal'),
    ('Simpler Model', 'exp_006', 0.009749, 'Better'),
    ('Even Simpler', 'exp_007', 0.009262, 'Better'),
    ('Ridge Regression', 'exp_008', 0.009192, 'Better'),
    ('GP Ensemble', 'exp_030', 0.008194, 'BEST CV'),
    ('XGB Ensemble', 'exp_041', 0.009002, 'Worse'),
    ('GroupKFold CV', 'exp_040', 0.009237, 'Worse'),
    ('Pure GP', 'exp_042', 0.014503, 'Much worse'),
    ('Aggressive Reg', 'exp_043', 0.009002, 'Same'),
    ('Stacking', 'exp_045', 0.008689, 'Worse'),
    ('CatBoost', 'exp_047', 0.013306, 'Worse'),
    ('RDKit', 'exp_048', 0.016324, 'Much worse'),
    ('GNN', 'exp_051', 0.014080, 'Much worse'),
    ('ChemBERTa', 'exp_052', 0.019444, 'Much worse'),
    ('Per-Target', 'exp_053', 0.009946, 'Worse'),
    ('Per-Solvent-Type', 'exp_054', 0.019519, 'Much worse'),
    ('Hyperparameter Opt', 'exp_055', 0.012658, 'Worse'),
    ('Advanced GNN', 'exp_056', 0.030013, 'Much worse'),
    ('Multi-Seed', 'exp_057', 0.009403, 'Worse'),
    ('Per-Target Weights', 'exp_058', 0.008701, 'Worse'),
    ('Physical Constraints', 'exp_059', 0.009611, 'Worse'),
    ('Spange Only', 'exp_060', 0.011265, 'Worse'),
    ('TabNet', 'exp_061', 0.036642, 'Much worse'),
    ('CQR', 'exp_062', 0.009899, 'Worse'),
]

print('\nExperiment Summary (62 experiments):')
print('='*60)
for name, exp_id, cv, result in experiments_tried:
    print(f'{exp_id}: {name:30s} CV={cv:.6f} ({result})')

In [ ]:
# What approaches haven't been tried?
print('\n=== APPROACHES NOT YET TRIED ===')
print()
print('1. DOMAIN-SPECIFIC APPROACHES:')
print('   - Reaction kinetics modeling (Arrhenius equation fitting)')
print('   - Solvent polarity-based weighting')
print('   - Temperature-dependent feature scaling')
print()
print('2. ENSEMBLE DIVERSITY:')
print('   - Negative correlation learning')
print('   - Snapshot ensembles')
print('   - Diverse feature subsets')
print()
print('3. CALIBRATION METHODS:')
print('   - Isotonic regression on predictions')
print('   - Temperature scaling')
print('   - Platt scaling')
print()
print('4. LOSS FUNCTION MODIFICATIONS:')
print('   - Focal loss (focus on hard examples)')
print('   - Asymmetric loss (penalize over/under-prediction differently)')
print('   - Weighted MSE by solvent similarity')
print()
print('5. DATA AUGMENTATION:')
print('   - Mixup between similar solvents')
print('   - Noise injection')
print('   - Synthetic data generation')

In [ ]:
# CRITICAL INSIGHT: The CV-LB gap is structural
print('\n=== CRITICAL INSIGHT ===')
print()
print('The CV-LB relationship has:')
print(f'  - Slope: {slope:.4f}')
print(f'  - Intercept: {intercept:.4f}')
print(f'  - R²: {r_value**2:.4f}')
print()
print('This means:')
print(f'  1. Even with CV=0, predicted LB = {intercept:.4f}')
print(f'  2. Target (0.0347) < Intercept ({intercept:.4f})')
print(f'  3. CV minimization alone CANNOT reach target')
print()
print('To reach target, we need to:')
print('  1. REDUCE the intercept (improve generalization)')
print('  2. OR find an approach with a different CV-LB relationship')
print()
print('The intercept represents the "base error" that exists regardless of CV.')
print('This could be due to:')
print('  - Distribution shift between train and test')
print('  - Model variance on unseen solvents')
print('  - Systematic bias in predictions')

In [ ]:
# What's special about the best generalizers?
print('\n=== BEST GENERALIZERS ANALYSIS ===')
print()
best_generalizers = df.nsmallest(3, 'residual_pct')
print('Top 3 best generalizers (lowest residual):')
for _, row in best_generalizers.iterrows():
    print(f"  {row['exp_id']}: CV={row['cv']:.6f}, LB={row['lb']:.4f}, residual={row['residual_pct']:.1f}%")

print()
print('What do they have in common?')
print('  - exp_000: Baseline MLP with Spange only')
print('  - exp_003: Spange + DRFP combined')
print('  - exp_005: Large ensemble (15 models)')
print()
print('Insight: Simpler models with Spange features generalize better!')
print('The best CV (exp_030 with GP ensemble) is NOT the best generalizer.')

In [ ]:
# Final recommendation
print('\n=== STRATEGIC RECOMMENDATION ===')
print()
print('Given:')
print(f'  - Best CV: 0.008194 (exp_030)')
print(f'  - Best LB: 0.0877 (exp_030)')
print(f'  - Target: 0.0347')
print(f'  - CV-LB intercept: {intercept:.4f} > Target')
print(f'  - 62 experiments tried, all follow same CV-LB relationship')
print()
print('The target IS reachable because:')
print('  1. The GNN benchmark achieved MSE 0.0039 (much better than target)')
print('  2. Other competitors may have found approaches we haven\'t tried')
print('  3. The CV-LB relationship is based on OUR experiments, not the true relationship')
print()
print('NEXT STEPS (Priority Order):')
print('  1. Try prediction calibration (isotonic regression, temperature scaling)')
print('  2. Try domain-specific loss functions (Arrhenius-weighted MSE)')
print('  3. Try ensemble diversity methods (negative correlation learning)')
print('  4. Try data augmentation (mixup, noise injection)')
print()
print('DO NOT SUBMIT unless we find an approach that could change the CV-LB relationship.')